# Fact Qualidade

Tabela fato de inspeções de qualidade e taxas de rejeição.

In [ ]:
# Configuração (lê parâmetros do job)
CATALOG = dbutils.widgets.get('catalog') if 'catalog' in [w.key for w in dbutils.widgets.getAll()] else 'manufatura_lakehouse'
SCHEMA_SILVER = dbutils.widgets.get('schema_silver') if 'schema_silver' in [w.key for w in dbutils.widgets.getAll()] else 'silver'
SCHEMA_GOLD = dbutils.widgets.get('schema_gold') if 'schema_gold' in [w.key for w in dbutils.widgets.getAll()] else 'gold'

def fqtn(schema, table):
    if CATALOG and CATALOG.lower() != 'none':
        return f"`{CATALOG}`.`{schema}`.`{table}`"
    else:
        return f"`{schema}`.`{table}`"

# Criar schema se necessário
if CATALOG and CATALOG.lower() != 'none':
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{CATALOG}`.`{SCHEMA_GOLD}`")
    spark.sql(f"USE CATALOG `{CATALOG}`")
else:
    spark.sql(f"CREATE DATABASE IF NOT EXISTS `{SCHEMA_GOLD}`")


In [ ]:
spark.sql(f"""
CREATE OR REPLACE TABLE {fqtn(SCHEMA_GOLD, 'fact_qualidade')} (
    inspection_id STRING,
    production_order_id STRING,
    equipment_id STRING,
    date_key INT,
    inspection_date TIMESTAMP,
    passed BOOLEAN,
    total_quantity INT,
    failed_quantity INT,
    rejection_rate_pct DOUBLE,
    defect_codes STRING
) USING DELTA
PARTITIONED BY (date_key)
""")

spark.sql(f"""
INSERT OVERWRITE {fqtn(SCHEMA_GOLD, 'fact_qualidade')}
SELECT 
    inspection_id,
    production_order_id,
    equipment_id,
    cast(date_format(inspection_date, 'yyyyMMdd') as int) as date_key,
    inspection_date,
    passed,
    total_quantity,
    failed_quantity,
    (failed_quantity / total_quantity) * 100 as rejection_rate_pct,
    defect_codes
FROM {fqtn(SCHEMA_SILVER, 'quality_inspections_clean')}
""")